## Strategic Approach to DP
### Framework for DP Problems
* first step is to define the state
  + A state is a set of variables taht can sufficiently describe a scenario
  + among a set of state variables, we only care about relevant ones
  + in Climbing Stairs (Leetcode 70), we only care about the current step. 
    + If i=6, we are describing the state of being on the 6th step
    + every unique value of i represents a unique state
    
#### The Framework
To solve a DP problem, we need to combine 3 things:
+ a function or data structure that will compute/contain the answer to the problem for every give state
  + For Climbing Stairs, let's say we have an function dp where dp(i) returns the number of ways to climb to the ith step. Solving the original problem would be as easy as return dp(n)
  + How did we decide on the design of the function? The problem is asking "How many distinct ways can you climb to the top?", so we decide that the function will represent how many distinct ways you can climb to a certain step - literally the original problem, but generalized for a given state.
  + Typically, top-down is implemented with a recursive function and hash map, whereas bottom-up is implemented with nested for loops and an array. When designing this function or array, we also need to decide on state variables to pass as arguments. This problem is very simple, so all we need to describe a state is to know what step we are currently on i
+ A recurrence relation to transition between states
  + A recurrence relation is an equation that relates different states with each other
  + Let's say that we needed to find how many ways we can climb to the 30th stair. Well, the problem states that we are allowed to take either 1 or 2 steps at a time. Logically, that means to climb to the 30th stair, we arrived from either the 28th or 29th stair. Therefore, the number of ways we can climb to the 30th stair is equal to the number of ways we can climb to the 28th stair plus the number of ways we can climb to the 29th stair.
  + The problem is, we don't know how many ways there are to climb to the 28th or 29th stair. However, we can use the logic from above to define a recurrence relation. In this case, 
    + dp(i) = dp(i - 1) + dp(i - 2)
    + As you can see, information about some states gives us information about other states
+ Base cases so that our recurrence relation doesn't go on infinitely
  + The equation dp(i) = dp(i - 1) + dp(i - 2) on its own will continue forever to negative infinity. WE need base cases so that the function will eventually return a actual number
  + Finding the base cases is often the easiest part of solving a DP problem, and just involves a little bit of logical thinking. When coming up with the base case(s) ask yourself: What state(s) can I find the answer to without using dynamic programming? In this example, we can reason that there is only 1 way to climb to the first stair (1 step once), and there are 2 ways to climb to the second stair (1 step twice and 2 steps once). Therefore, our base cases are
    + dp(1) = 1
    + dp(2) = 2 
    

#### Example Implementations 
* Top-down implementation using memoization
  + Time complexity:
  + O(n) by using memoization
  + we use hashmap instead of array because
    + some DP problems will require one, and they're hassle-free to use as you don't need to worry about sizing an array correctly
    + when using top-down DP, some problems do not require us to solve every single subproblem, in which case an array may use more memory than a hashmap

In [4]:
class Solution:
    def climbStairs(self, n: int) -> int:
        def dp(i):
            if i <= 2: 
                return i
            if i not in memo:
                # Instead of just returning dp(i - 1) + dp(i - 2), calculate it once and then
                # store the result inside a hashmap to refer to in the future.
                memo[i] = dp(i - 1) + dp(i - 2)
            
            return memo[i]
        
        memo = {}
        return dp(n)

* Bottom-up implementation using array as a common implementation
  + Time complexity and space complexity:
  + O(n)  

In [5]:
class Solution:
    def climbStairs(self, n: int) -> int:
        if n == 1:
            return 1
            
        # An array that represents the answer to the problem for a given state
        dp = [0] * (n + 1)
        dp[1] = 1 # Base cases
        dp[2] = 2 # Base cases
        
        for i in range(3, n + 1):
            dp[i] = dp[i - 1] + dp[i - 2] # Recurrence relation

        return dp[n]

#### To Summarize
* with DP problems, we can use logical thinking to find the answer to teh original problem for certain inputs, in this case,we reason that there is 1 way to climb to the first stair and 2 ways to climb to the second stair
* we then use recurrence relation to find the answer to the original problem for any state, in this case, for any stair number
* Finding the recurrence relation involves thining about how moving from one stage to another changes the answer to the problem

### Example Leetcode 198 House Robber
* The robber with traverse from house 0 to the last house (n-1). There are two options when arriving at each house
  + not rob the current house, the money will be the max(not rob the previous house, rob the previous house)
  + rob the current house, the money will be (not rob the previous house + current value)
  + Therefore, for each house, the rob and not_rob corresponding to the max value can get by robbing and not_robbing the current house, respectively
  
* how to apply the framework
  + define a function or array that answers the problem for a given state
  + we define the state as the index of the house that can alone define the value we can get upto this house
    + If the problem had an added constraint such as "you are only allowed to rob up to k houses", then k would be another necessary state variable. This is because being at, say house 4 with 3 robberies left is different than being at house 4 with 5 robberies left
  + a recurrence relation to transition between states
    + a good place to start is to think about a general state (in this case, let's say we're at the house at index i, and use information from the problem description to think about how other states relate to the current one
    + If we decide not to rob the house, then we don't gain any money. Whatever money we had from the previous house is how much money we will have at this house - which is dp(i - 1)
    + If we decide to rob the house, then we gain nums\[i\] money. However, this is only possible if we did not rob the previous house. This means the money we had when arriving at this house is the money we had from the previous house without robbing it, which would be however much money we had 2 houses ago, dp(i - 2). After robbing the current house, we will have dp(i - 2) + nums\[i\]
    + From these two options, we always want to pick the one that gives us maximum profits. Putting it together, we have our recurrence relation: dp(i) = max(dp(i - 1), dp(i - 2) + nums\[i\])
* note that the dp(i-1) is the max of rob and not_rob at index i-1, so it might be the same as dp(i-2). It will be more clear to return the rob, not_rob for each index, both for top down and bottom up  
```python
class Solution:
    # bottom up
    def rob(self, nums: List[int]) -> int:
        if not nums: 
            return 0
        
        if len(nums) < 3:
            return max(nums)
        
        prev_two, prev_one = nums[0], max(nums[0], nums[1])
        
        for i in range(2, len(nums)):
            prev_two, prev_one = prev_one, max(prev_two + nums[i], prev_one)
            
        return prev_one
    ```

```python
class Solution:
    # top down
    def rob(self, nums: List[int]) -> int:
        if not nums: 
            return 0
        
        if len(nums) < 3:
            return max(nums)
        
        @lru_cache
        def rob_helper(index: int) -> int:
            if index == 0:
                return nums[0]
            if index == 1:
                return max(nums[1], nums[0])
            return max(rob_helper(index-1), rob_helper(index-2)+ nums[index])
        
        return rob_helper(len(nums)-1)
```    

In [7]:
from typing import List
# top down
class Solution:
    # top down    
    def rob(self, nums: List[int]) -> int:
        # top down
        
        if not nums:
            return 0        
        
        # return a tuple of rob and not rob up to each house
        @lru_cache
        def rob_helper(house_index: int) -> [int, int]:
            if house_index == 0:
                return nums[0], 0
            
            rob, not_rob = rob_helper(house_index - 1)            
            return not_rob + nums[house_index], max(rob, not_rob)
        
        n = len(nums)
        
        return max(rob_helper(n-1))
    
# bottom up
class Solution:
    # bottom up
    def rob(self, nums: List[int]) -> int:
        if not nums: 
            return 0
        
        if len(nums) < 3:
            return max(nums)
        
        rob, not_rob = nums[0], 0
        
        for i in range(1, len(nums)):
            rob, not_rob = not_rob + nums[i], max(rob, not_rob)
            
        return max(rob, not_rob)         

### Leetcode 746 Min Cost Climbing Stairs
* the state variable is the cost to arrive at the ith stair
* note that the question asks the cost to arrive at the top of the ith stair, where i is the last element of the cost
* start from 2, to n where n is the length of the cost array (for i in range(2, n+1))

In [9]:
from typing import List

# bottom up
class Solution:
    def minCostClimbingStairs(self, cost: List[int]) -> int:
        n = len(cost)
        
        prev_1, prev_2 = 0, 0
        
        for i in range(2, n+1):
            prev_2, prev_1 = prev_1, min(prev_2+cost[i-2], prev_1+cost[i-1])
            
        return prev_1 
    
# top down
class Solution:
    def minCostClimbingStairs(self, cost: List[int]) -> int:
        n = len(cost)
        
        @lru_cache
        def cost_helper(i: int)-> int:
            if i == 0 or i == 1:
                return 0
            return min(cost_helper(i-1) + cost[i-1], cost_helper(i-2) + cost[i-2])
        
        return cost_helper(n)

### Leetcode 1137 Tribonacci Number
* set up the prev_1, prev_2 and prev_3 as 1, 1, 0
* follow the definition to update the prev_1, prev_2 and prev_3
* output prev_1

In [ ]:
from typing import List
# bottom up
class Solution:
    def tribonacci(self, n: int) -> int:
        
        if n ==0:
            return 0
        if n == 1 or n == 2:
            return 1
        
        prev_3, prev_2, prev_1 = 0, 1, 1
        
        for i in range(3, n+1):
            prev_3, prev_2, prev_1 = prev_2, prev_1, prev_1 + prev_2 + prev_3
            
        return prev_1  
    
# top down
class Solution:
    def tribonacci(self, n: int) -> int:
        
        @lru_cache
        def trib_helper(n :int) -> int:
            if n == 0:
                return 0
            if n == 1 or n == 2:
                return 1
            
            return trib_helper(n-3) + trib_helper(n-2) + trib_helper(n-1)
        
        return trib_helper(n)

### Leetcode 740 Delete and Earn
* give an integer array, you can pick up a number, and gain that number, but have to delete num -1 and num+1 from the array, meaning that you will not gain those deleted numbers, and select the max gain you can get
* state variable is the unique num value, and corresponding to each state variable value, we calculate the gain we can get
  + the typical DP scenario, we eithe take the num, or not
  + if we take a num, i, we gain all the sum of all of is in the array, but the max gain we can get beside that, is the dp(i-2), since i-1 is deleted.
  + we don't consider the fact that i+1 is removed from the list, since we will return dp(max_value_of_array)
* we store each state variable (the unique num in array) with the total value of this variable in array in a dictionary
* we sort the keys in the dictionary and traverse from the frist key(smallest key)
* we use two variables, back_one and back_two to track the state transition.
  + back_one and back_two corresponding to the gains obtained by the keys one unit and two unit before the current key, respectively
  + for each state with a value of num
    + the new back_two = back_one
    + if the key just before it equals num-1, we assign the max of taking the num, max(num + back_two, and back_one) to the new back_one
    + if the key just before it is less than num-1, we can always take the num and combine it with back_one to get more gain and assign it to the new back_one 

In [10]:
from typing import List

class Solution:
    def deleteAndEarn(self, nums: List[int]) -> int:
        if len(nums) == 1:
            return nums[0]
        
        num_counter = defaultdict(int)
        
        for num in nums:
            num_counter[num] += num
            
        keys = sorted(num_counter.keys())  
        
        back_two, back_one = 0, num_counter[keys[0]]
        
        for i in range(1, len(keys)):
            key = keys[i]
            if key == keys[i-1] + 1:
                back_two, back_one = back_one, max(back_two + num_counter[key], back_one)
            else:
                back_two, back_one = back_one, back_one + num_counter[key]
        
        return back_one        

### Multidimensional DP
* The following are common things to look out for in DP problems that require a state variable:
  + An index along some input. This is usually used if an input is given as an array or string. This has been the sole state variable for all the problems that we've looked at so far, and it has represented the answer to the problem if the input was considered only up to that index - for example, if the input is 
nums = \[0, 1, 2, 3, 4, 5, 6\] then dp(4) would represent the answer to the problem for the input nums = \[0, 1, 2, 3, 4\]
  + A second index along some input. Sometimes, you need two index state variables, say i and j. In some questions, these variables represent the answer to the original problem if you considered the input starting at index i and ending at index 
j. Using the same example above, dp(1, 3) would solve the problem for the input nums = \[1, 2, 3\], if the original input was \[0, 1, 2, 3, 4, 5, 6\].
  + Explicit numerical constraints given in the problem. For example, "you are only allowed to complete k transactions", or "you are allowed to break up to k obstacles", etc.
  + Variables that describe statuses in a given state. For example "true if currently holding a key, false if not", "currently holding k packages" etc.
  + Some sort of data like a tuple or bitmask used to indicate things being "visited" or "used". For example, "bitmask is a mask where the ith bit indicates if the ith city has been visited". Note that mutable data structures like arrays cannot be used - typically, only immutable data structures like numbers and strings can be hashed, and therefore memoized.

### Top-down to Bottom-up
* Steps to convert a top-down into bottom-up
  1. Start with a completed top-down implementation
  2. Initialize an array dp that is sized according to your state variables. For example, let's say the input to the problem was an array nums and an integer k that represents the maximum number of actions allowed. Your array dp would be 2D with one dimension of length nums.length and the other of lengthk. The values should be initialized as some default value opposite of what the problem is asking for. For example, if the problem is asking for the maximum of something, set the values to negative infinity. If it is asking for the minimum of something, set the values to infinity
  3. Set your base cases, same as the ones you are using in your top-down function. Recall in House Robber, dp(0) = nums\[0\] and dp(1) = max(nums\[0\], nums\[1\]). In bottom-up, dp\[0\] = nums\[0\] and dp\[1\] = max(nums\[0\], nums\[1\])
  4. Write a for-loop(s) that iterate over your state variables. If you have multiple state variables, you will need nested for-loops. These loops should start iterating from the base cases.

  5. Now, each iteration of the inner-most loop represents a given state, and is equivalent to a function call to the same state in top-down. Copy the logic from your function into the for-loop and change the function calls to accessing your array. All dp(...) changes into dp\[...\]
  6. We're done! dp is now an array populated with the answer to the original problem for all possible states. Return the answer to the original problem, by changing return dp(...) to return dp\[...\]

In [ ]:
# top-down 
class Solution:
    def rob(self, nums: List[int]) -> int:

        def dp(i: int) -> int:
            # Base cases
            if i == 0:
                return nums[0]
            elif i == 1:
                return max(nums[0], nums[1])
            
            if i not in memo:
                # Use recurrence relation to calculate dp[i].
                memo[i] = max(dp(i - 1), dp(i - 2) + nums[i])
            
            return memo[i]
        
        memo = {}
        return dp(len(nums) - 1)
    
# convert to bottom up
class Solution:
    def rob(self, nums: List[int]) -> int:   
        n = len(nums)
        if n == 1:
            return nums[0]
        dp = [0] * n
        
        #Base Cases
        dp[0] = nums[0]
        dp[1] = max(nums[0], nums[1])
        
        for i in range(2, n):
            # Use recurrence relation to calculate dp[i].
            dp[i] = max(dp[i - 1], dp[i - 2] + nums[i])
        
        return dp[n - 1]

### Leetcode 1770 Maximum Score from Performing Multiplication Operations
* problem overview
  + we have two arrays, nums and multipliers whose lengths are n and m, respectively
  + we have m operations that will use all element of multipliers in m steps
  + in each step, we select either the leftmost or the rightmost element from nums, multiply it by the multiplier\[i\]
  + we need to find the max values of the sum of these steps
* implementations
  + top down
    + we set up two state variables, i, which refers to how many steps for the current step (eg. the ith step)
      + we define the value of the corresponding mutlipliers as mult = multipliers\[i\]
    + left, the index of the leftmost element in num for the current step
    + we can calculate the rightmost element index as n-1-(i-left)
    + we define the recursive function of def score(i, left)
    + for a general ith step, score(i, left) returns max(mult * nums\[left\] + score(i+1, left+1), mult * nums\[right\] + score(i+1, left))
    + base case is when i==m, returns 0
* bottom up
  + this is a typical 2-dimensional DP problem
  + the first dimension is the number of operations for the current step (ith step means i-1 operations have completed, and this is the ith operation)
  + the second dimension is the left index. this index is restricted by the number of operations, meaning that if all the previous i-1 operations have choosen the left end, then the current left index is i, which is the max of left at this given step at i
  + we start the first dimension from m-1, because we knwo the base cases are when i == m, then no matter what left index is, the results are always zero
  + on the second dimesion, we iterate from 0 to i, or the opposite direction. We only need to use the values in the column right to it. using the transition function of 
    + max(dp\[i+1\]\[left\] + mult * nums\[right\], dp\[i+1\]\[left+1\] + mult*nums\[left\]
  + return dp\[0\]\[0\]  

In [ ]:
# top-down
class Solution:
    def maximumScore(self, nums: List[int], multipliers: List[int]) -> int:
        
        # top down
        
        n, m = len(nums), len(multipliers)
        
        @lru_cache
        def score(i: int, left: int) -> int:
            if i == m:
                return 0
            
            right = n-1 - (i-left)
            mult = multipliers[i]
            return max(mult * nums[left] + score(i+1, left+1), mult * nums[right] + score(i+1, left))
        
        return score(0, 0)

# bottom-up
class Solution:
    def maximumScore(self, nums: List[int], multipliers: List[int]) -> int:
        
        # bottom up
        
        n, m = len(nums), len(multipliers)
        
        dp = [[0] * (m+1) for _ in range(m+1)]
        
        for i in range(m-1, -1, -1):
            mult = multipliers[i]
            for left in range(i+1):
                right = n-1-(i-left)
                dp[i][left] = max(dp[i+1][left] + mult * nums[right], dp[i+1][left+1]+ mult*nums[left])
                
        return dp[0][0]               
        